# Tests RAMAdvisor - API Gemini avec Système d'Atypicité V3

Ce notebook est organisé en 3 parties :
1. **Test de la clé API** - Validation de la connexion Gemini
2. **Test normal** - Requête standard avec prompt v3
3. **Jeu d'essai** - 5 profils clients différents pour tester le score d'atypicité

## Partie 1 : Test de la Clé API

In [ ]:
# Installer la librairie Gemini (Google Generative AI)
!pip install --upgrade google-generativeai

In [4]:
# Tester l'API Gemini avec ta clé API
import google.generativeai as genai
import os

API_KEY = os.getenv('GEMINI_API_KEY')
if not API_KEY:
    # Demander la clé manuellement (masquée si possible) et la conserver pour la session
    try:
        from getpass import getpass
        API_KEY = getpass("GEMINI_API_KEY non définie — entrez la clé (saisie masquée) : ")
    except Exception:
        API_KEY = input("GEMINI_API_KEY non définie — entrez la clé : ")
    if not API_KEY:
        raise RuntimeError("GEMINI_API_KEY doit être fournie (définir la variable d'environnement ou la saisir manuellement).")
    # stocker pour la session (ne modifie pas les variables système de façon permanente)
    os.environ['GEMINI_API_KEY'] = API_KEY

genai.configure(api_key=API_KEY)

# Exemple de génération de texte
model_light = genai.GenerativeModel("gemini-2.0-flash-lite")
model_experimental = genai.GenerativeModel("learnlm-2.0-flash-experimental")
model_pro = genai.GenerativeModel("gemini-2.5-pro")

response = model_light.generate_content('Dis bonjour en français !')
print(response.text)

Bonjour !



## Partie 2 : Test Normal - Requête Standard avec Prompt V3

In [5]:
# Test normal - Client standard avec objectif typique
import google.generativeai as genai
import os

# Configuration de l'API (utiliser variable d'environnement obligatoire)
API_KEY = os.getenv('GEMINI_API_KEY')
if not API_KEY:
    raise RuntimeError('GEMINI_API_KEY not set in environment')

genai.configure(api_key=API_KEY)

# Client test standard
client_standard = {
    "objectif": "Je souhaite préparer ma retraite en constituant un patrimoine pour compléter ma pension",
    "profil_risque": "Équilibré",
    "montant_initial": "20000€",
    "montant_mensuel": "600€",
    "horizon": "15 ans"
}

print("=== TEST NORMAL - CLIENT STANDARD ===")
for key, value in client_standard.items():
    print(f"{key}: {value}")

# Charger le template v3 et la knowledge base
with open('prompt_template_v3.md', 'r', encoding='utf-8') as f:
    prompt_template_v3 = f.read()

with open('knowledge_base.txt', 'r', encoding='utf-8') as f:
    knowledge = f.read()

=== TEST NORMAL - CLIENT STANDARD ===
objectif: Je souhaite préparer ma retraite en constituant un patrimoine pour compléter ma pension
profil_risque: Équilibré
montant_initial: 20000€
montant_mensuel: 600€
horizon: 15 ans


## Partie 3 : Jeu d'Essai - 5 Profils Clients Différents

Test du système d'atypicité avec des objectifs variés pour valider les scores 1-10 et l'adaptation de l'IA.

In [3]:
# Jeu d'essai - 5 clients avec différents niveaux d'atypicité
import google.generativeai as genai
import os

# Configuration API
API_KEY = os.getenv('GEMINI_API_KEY')
if not API_KEY:
    raise RuntimeError('GEMINI_API_KEY not set in environment')

genai.configure(api_key=API_KEY)

# 5 profils clients avec objectifs de différents niveaux d'atypicité
clients_test = [
    {
        "nom": "Client 1 - Ultra-typique (Score attendu: 9-10)",
        "objectif": "Préparer ma retraite en constituant un patrimoine diversifié",
        "profil_risque": "Équilibré",
        "montant_initial": "25000€",
        "montant_mensuel": "500€",
        "horizon": "20 ans"
    },
    {
        "nom": "Client 2 - Typique avec nuance (Score attendu: 7-8)",
        "objectif": "Constituer un patrimoine pour ma retraite en privilégiant l'immobilier et l'investissement responsable",
        "profil_risque": "Prudent",
        "montant_initial": "30000€",
        "montant_mensuel": "800€",
        "horizon": "18 ans"
    },
    {
        "nom": "Client 3 - Moyennement typique (Score attendu: 5-6)",
        "objectif": "Financer les études de mes enfants tout en préparant ma retraite avec une approche équilibrée",
        "profil_risque": "Équilibré",
        "montant_initial": "15000€",
        "montant_mensuel": "600€",
        "horizon": "12 ans"
    },
    {
        "nom": "Client 4 - Peu typique (Score attendu: 3-4)",
        "objectif": "Investir uniquement dans des actifs éthiques et durables pour financer mon tour du monde dans 8 ans",
        "profil_risque": "Audacieux",
        "montant_initial": "40000€",
        "montant_mensuel": "1200€",
        "horizon": "8 ans"
    },
    {
        "nom": "Client 5 - Ultra-atypique (Score attendu: 1-2)",
        "objectif": "Générer des revenus passifs maximaux pour devenir financièrement indépendant et quitter mon emploi dans 5 ans",
        "profil_risque": "Audacieux",
        "montant_initial": "100000€",
        "montant_mensuel": "3000€",
        "horizon": "5 ans"
    }
]

# Fonctions utilitaires
def load_templates_and_knowledge():
    with open('prompt_template_v3.md', 'r', encoding='utf-8') as f:
        template = f.read()
    with open('knowledge_base.txt', 'r', encoding='utf-8') as f:
        knowledge = f.read()
    return template, knowledge

def filter_knowledge_by_risk(profil_risque, knowledge_content):
    risk_objectives = {
        "Prudent": ["PRESERVATION", "REVENU"],
        "Équilibré": ["REVENU", "CROISSANCE_MODEREE"], 
        "Audacieux": ["CROISSANCE", "CROISSANCE_AGGRESSIVE"]
    }
    
    relevant_objectives = risk_objectives.get(profil_risque, ["CROISSANCE_MODEREE"])
    sections = knowledge_content.split("OBJECTIF :")
    filtered = []
    
    for section in sections[1:]:
        section_content = "OBJECTIF :" + section
        for objective in relevant_objectives:
            if objective in section_content:
                filtered.append(section_content)
                break
        if len(filtered) >= 2:
            break
    
    return "\n\n".join(filtered)

def test_client(client_data, template, knowledge):
    print(f"\n{'='*60}")
    print(f"TEST: {client_data['nom']}")
    print(f"{'='*60}")
    print(f"Objectif: {client_data['objectif']}")
    print(f"Profil: {client_data['profil_risque']}")
    print(f"Montant initial: {client_data['montant_initial']}")
    print(f"Épargne mensuelle: {client_data['montant_mensuel']}")
    print(f"Horizon: {client_data['horizon']}")
    
    # Préparation du prompt
    filtered_knowledge = filter_knowledge_by_risk(client_data["profil_risque"], knowledge)
    personalized_prompt = template.format(**client_data)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}"
    
    print(f"\nLongueur prompt: {len(final_prompt)} caractères")
    
    # Appel Gemini
    model = genai.GenerativeModel("gemini-2.0-flash-lite")
    response = model.generate_content(final_prompt)
    
    # Extraction du score d'atypicité de la réponse (si présent)
    response_text = response.text
    if "Score attribué" in response_text:
        lines = response_text.split('\n')
        for line in lines:
            if "Score attribué" in line:
                print(f"✅ {line.strip()}")
                break
    
    print(f"\n📊 RÉPONSE (premiers 400 caractères):")
    print(response_text[:400] + "..." if len(response_text) > 400 else response_text)
    print(f"\n🔍 Recherche de mots-clés d'adaptation:")
    
    # Analyse rapide du niveau d'adaptation
    adaptation_keywords = ["strictement", "adapter", "personnalisé", "dévier", "ajuster"]
    found_keywords = [kw for kw in adaptation_keywords if kw.lower() in response_text.lower()]
    if found_keywords:
        print(f"Mots-clés trouvés: {', '.join(found_keywords)}")
    
    return response_text

# Exécution des tests
print("🚀 DÉBUT DU JEU D'ESSAI - 5 CLIENTS DIFFÉRENTS")
print("Objectif: Valider le système de score d'atypicité et l'adaptation de l'IA")

template, knowledge = load_templates_and_knowledge()

# Test de chaque client
results = []
for client in clients_test:
    try:
        result = test_client(client, template, knowledge)
        results.append({"client": client["nom"], "success": True, "response_length": len(result)})
    except Exception as e:
        print(f"❌ ERREUR pour {client['nom']}: {str(e)}")
        results.append({"client": client["nom"], "success": False, "error": str(e)})

# Résumé final
print(f"\n{'='*60}")
print("📋 RÉSUMÉ DU JEU D'ESSAI")
print(f"{'='*60}")
for result in results:
    status = "✅ OK" if result["success"] else "❌ ERREUR"
    length = f"({result.get('response_length', 0)} car.)" if result["success"] else ""
    print(f"{status} {result['client']} {length}")

print(f"\n🎯 Tests réussis: {sum(1 for r in results if r['success'])}/{len(results)}")
print("✅ JEU D'ESSAI TERMINÉ")

🚀 DÉBUT DU JEU D'ESSAI - 5 CLIENTS DIFFÉRENTS
Objectif: Valider le système de score d'atypicité et l'adaptation de l'IA

TEST: Client 1 - Ultra-typique (Score attendu: 9-10)
Objectif: Préparer ma retraite en constituant un patrimoine diversifié
Profil: Équilibré
Montant initial: 25000€
Épargne mensuelle: 500€
Horizon: 20 ans

Longueur prompt: 9197 caractères
✅ **Score attribué : 9/10**

📊 RÉPONSE (premiers 400 caractères):
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée

Bonjour ! Félicitations pour votre démarche proactive visant à préparer votre retraite en constituant un patrimoine diversifié. Votre objectif, associé à votre profil de risque équilibré, à votre capital initial de 25 000€ et à votre capacité d'épargne mensuelle de 500€, nous permet d'établir une stratégie d'investi...

🔍 Recherche de mots-clés d'adaptation:
Mots-clés trouvés: personnalisé

TEST: Client 2 - Typique avec nuance (Score attendu: 7-8)
Objectif: Constituer un patrimoine pou